In [ ]:
# Google Colab Setup / Google Colab セットアップ
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import sys
    from pathlib import Path
    
    proj_root = Path("/content/drive/MyDrive/image_processing_tutorial-main")
    if str(proj_root) not in sys.path:
        sys.path.insert(0, str(proj_root))
    
    print("Google Colab environment detected and configured!")
    print("Google Colab環境が検出され、設定されました！")
else:
    print("Local Jupyter environment detected")
    print("ローカルJupyter環境が検出されました")



This tutorial demonstrates nonlinear classification using multilayer perceptrons implemented with PyTorch. We'll compare linear and nonlinear models on a sinusoidal boundary classification problem.

このチュートリアルでは、PyTorchで実装した多層パーセプトロンを使用した非線形分類を実演します。正弦波境界分類問題で線形モデルと非線形モデルを比較します。

- Understand limitations of linear models / 線形モデルの限界を理解
- Implement MLP using PyTorch / PyTorchを使用してMLPを実装
- Compare linear vs nonlinear classification / 線形vs非線形分類の比較
- Visualize decision boundaries / 決定境界の可視化
- Analyze model performance / モデル性能の解析




**Question:** How do we set up PyTorch for deep learning and create a challenging nonlinear classification problem?

**質問:** 深層学習のためにPyTorchをセットアップし、挑戦的な非線形分類問題を作成するにはどうすればよいですか？

**Answer:** We'll import PyTorch and related libraries, then generate data with a sinusoidal decision boundary that linear models cannot handle effectively.

**回答:** PyTorchと関連ライブラリをインポートし、線形モデルでは効果的に処理できない正弦波決定境界を持つデータを生成します。


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns

np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"使用デバイス: {device}")

print("Libraries imported successfully / ライブラリのインポートが完了しました")
print("PyTorch version:", torch.__version__)


In [ ]:

N = 500  # Number of samples / サンプル数

x = np.random.uniform(low=-1, high=1, size=(N, 2))
y = np.zeros(N)

y[x[:, 1] - np.sin(2*np.pi*x[:, 0]) > 0] = 1

plt.figure(figsize=(10, 8))
plt.plot(x[y==0, 0], x[y==0, 1], 'bo', alpha=0.6, label='Class 0', markersize=6)
plt.plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.6, label='Class 1', markersize=6)

xx = np.linspace(-1, 1, 200)
yy = np.sin(2*np.pi*xx)
plt.plot(xx, yy, 'g-', linewidth=3, label='True Boundary')

plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Nonlinear Classification Problem: Sinusoidal Boundary')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Generated {N} samples with sinusoidal decision boundary")
print(f"正弦波決定境界を持つ{N}個のサンプルを生成しました")
print(f"Class 0: {np.sum(y==0)} samples, Class 1: {np.sum(y==1)} samples")
print(f"クラス0: {np.sum(y==0)}サンプル、クラス1: {np.sum(y==1)}サンプル")

X_tensor = torch.FloatTensor(x).to(device)
y_tensor = torch.FloatTensor(y).to(device)

print("Data converted to PyTorch tensors / データをPyTorchテンソルに変換しました")




**Question:** How do we create a linear baseline model using PyTorch to understand the limitations of linear classification?

**質問:** 線形分類の限界を理解するために、PyTorchを使用して線形ベースラインモデルを作成するにはどうすればよいですか？

**Answer:** We'll implement a simple linear model with one fully connected layer and sigmoid activation, then train it to see how it performs on the nonlinear problem.

**回答:** 1つの全結合層とシグモイド活性化を持つシンプルな線形モデルを実装し、非線形問題でのパフォーマンスを確認するために訓練します。


In [ ]:

class LinearModel(nn.Module):
    def __init__(self, input_dim=2):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

linear_model = LinearModel().to(device)
linear_criterion = nn.BCELoss()
linear_optimizer = optim.Adam(linear_model.parameters(), lr=0.01)

print("Linear model architecture:")
print("線形モデルアーキテクチャ:")
print(linear_model)

epochs = 1000
batch_size = 100

dataset = TensorDataset(X_tensor, y_tensor.unsqueeze(1))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Training linear model for {epochs} epochs...")
print(f"線形モデルを{epochs}エポック訓練中...")


In [ ]:

linear_losses = []
linear_accuracies = []

linear_model.train()
for epoch in range(epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    for batch_x, batch_y in dataloader:
        linear_optimizer.zero_grad()
        outputs = linear_model(batch_x)
        loss = linear_criterion(outputs, batch_y)
        loss.backward()
        linear_optimizer.step()
        
        epoch_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_loss = epoch_loss / len(dataloader)
    accuracy = correct / total
    linear_losses.append(avg_loss)
    linear_accuracies.append(accuracy)
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}")
        print(f"エポック {epoch+1}: 損失 = {avg_loss:.4f}, 精度 = {accuracy:.4f}")

print("Linear model training completed!")
print("線形モデルの訓練が完了しました！")

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(linear_losses, 'b-', linewidth=2)
plt.title('Linear Model: Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(linear_accuracies, 'r-', linewidth=2)
plt.title('Linear Model: Accuracy Evolution')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

final_linear_loss = linear_losses[-1]
final_linear_accuracy = linear_accuracies[-1]
print(f"Linear model final loss: {final_linear_loss:.4f}")
print(f"Linear model final accuracy: {final_linear_accuracy:.4f}")
print(f"線形モデル最終損失: {final_linear_loss:.4f}")
print(f"線形モデル最終精度: {final_linear_accuracy:.4f}")




**Question:** How do we implement a multilayer perceptron in PyTorch to handle nonlinear classification problems?

**質問:** 非線形分類問題を処理するために、PyTorchで多層パーセプトロンを実装するにはどうすればよいですか？

**Answer:** We'll create an MLP with hidden layers and ReLU activation functions, which can learn complex nonlinear decision boundaries.

**回答:** 隠れ層とReLU活性化関数を持つMLPを作成し、複雑な非線形決定境界を学習できるようにします。


In [ ]:

class MLPModel(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=5, output_dim=1):
        super(MLPModel, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

mlp_model = MLPModel().to(device)
mlp_criterion = nn.BCELoss()
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.01)

print("MLP Model Architecture:")
print("MLPモデルアーキテクチャ:")
print(mlp_model)

total_params = sum(p.numel() for p in mlp_model.parameters())
trainable_params = sum(p.numel() for p in mlp_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"総パラメータ数: {total_params}")
print(f"訓練可能パラメータ数: {trainable_params}")

print("MLP model built successfully!")
print("MLPモデルの構築が完了しました！")




**Question:** How do we train the MLP model and compare its performance with the linear baseline?

**質問:** MLPモデルを訓練し、線形ベースラインとの性能を比較するにはどうすればよいですか？

**Answer:** We'll train the MLP for more epochs and compare loss evolution, accuracy, and final performance metrics.

**回答:** MLPをより多くのエポックで訓練し、損失の変化、精度、最終性能指標を比較します。


In [ ]:

mlp_epochs = 2000
mlp_losses = []
mlp_accuracies = []

print(f"Training MLP model for {mlp_epochs} epochs...")
print(f"MLPモデルを{mlp_epochs}エポック訓練中...")

mlp_model.train()
for epoch in range(mlp_epochs):
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    for batch_x, batch_y in dataloader:
        mlp_optimizer.zero_grad()
        outputs = mlp_model(batch_x)
        loss = mlp_criterion(outputs, batch_y)
        loss.backward()
        mlp_optimizer.step()
        
        epoch_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_loss = epoch_loss / len(dataloader)
    accuracy = correct / total
    mlp_losses.append(avg_loss)
    mlp_accuracies.append(accuracy)
    
    if (epoch + 1) % 400 == 0:
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}")
        print(f"エポック {epoch+1}: 損失 = {avg_loss:.4f}, 精度 = {accuracy:.4f}")

print("MLP model training completed!")
print("MLPモデルの訓練が完了しました！")


In [ ]:

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(linear_losses, 'b-', linewidth=2, label='Linear Model')
plt.plot(mlp_losses, 'r-', linewidth=2, label='MLP Model')
plt.title('Loss Evolution Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale for better visualization / より良い可視化のための対数スケール

plt.subplot(1, 3, 2)
plt.plot(linear_accuracies, 'b-', linewidth=2, label='Linear Model')
plt.plot(mlp_accuracies, 'r-', linewidth=2, label='MLP Model')
plt.title('Accuracy Evolution Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

models = ['Linear', 'MLP']
final_losses = [linear_losses[-1], mlp_losses[-1]]
final_accuracies = [linear_accuracies[-1], mlp_accuracies[-1]]

plt.subplot(1, 3, 3)
x_pos = np.arange(len(models))
plt.bar(x_pos, final_accuracies, alpha=0.7, color=['blue', 'red'])
plt.xlabel('Model Type')
plt.ylabel('Final Accuracy')
plt.title('Final Accuracy Comparison')
plt.xticks(x_pos, models)
plt.grid(True, alpha=0.3)

for i, acc in enumerate(final_accuracies):
    plt.text(i, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n" + "="*50)
print("MODEL PERFORMANCE COMPARISON / モデル性能比較")
print("="*50)
print(f"Linear Model - Loss: {final_losses[0]:.4f}, Accuracy: {final_accuracies[0]:.4f}")
print(f"MLP Model    - Loss: {final_losses[1]:.4f}, Accuracy: {final_accuracies[1]:.4f}")
print(f"線形モデル - 損失: {final_losses[0]:.4f}, 精度: {final_accuracies[0]:.4f}")
print(f"MLPモデル  - 損失: {final_losses[1]:.4f}, 精度: {final_accuracies[1]:.4f}")
print("="*50)




**Question:** How can we visualize and compare the decision boundaries learned by linear and MLP models?

**質問:** 線形モデルとMLPモデルが学習した決定境界をどのように可視化し比較できますか？

**Answer:** We'll create prediction grids and visualize the decision boundaries, comparing them with the true sinusoidal boundary.

**回答:** 予測グリッドを作成し決定境界を可視化し、真の正弦波境界と比較します。


In [ ]:

def create_meshgrid(resolution=128):
    """Create meshgrid for visualization / 可視化用メッシュグリッド作成"""
    xx = np.linspace(-1, 1, resolution)
    yy = np.linspace(-1, 1, resolution)
    XX, YY = np.meshgrid(xx, yy)
    grid_points = np.hstack((XX.reshape(-1, 1), YY.reshape(-1, 1)))
    return XX, YY, grid_points

XX, YY, grid_points = create_meshgrid(128)
grid_tensor = torch.FloatTensor(grid_points).to(device)

linear_model.eval()
mlp_model.eval()

with torch.no_grad():
    linear_predictions = linear_model(grid_tensor).cpu().numpy().reshape(128, 128)
    mlp_predictions = mlp_model(grid_tensor).cpu().numpy().reshape(128, 128)

print("Decision boundary predictions generated")
print("決定境界予測が生成されました")


In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].contourf(XX, YY, linear_predictions, levels=50, alpha=0.6, cmap='RdYlBu')
axes[0, 0].plot(x[y==0, 0], x[y==0, 1], 'bo', alpha=0.6, markersize=3)
axes[0, 0].plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.6, markersize=3)
axes[0, 0].set_title('Linear Model Decision Boundary')
axes[0, 0].set_xlabel('X1')
axes[0, 0].set_ylabel('X2')
axes[0, 0].grid(True, alpha=0.3)

im = axes[0, 1].contourf(XX, YY, mlp_predictions, levels=50, alpha=0.6, cmap='RdYlBu')
axes[0, 1].plot(x[y==0, 0], x[y==0, 1], 'bo', alpha=0.6, markersize=3)
axes[0, 1].plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.6, markersize=3)
axes[0, 1].set_title('MLP Model Decision Boundary')
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].grid(True, alpha=0.3)

plt.colorbar(im, ax=axes[0, 1], label='Prediction Probability')

xx_true = np.linspace(-1, 1, 200)
yy_true = np.sin(2*np.pi*xx_true)

axes[1, 0].plot(x[y==0, 0], x[y==0, 1], 'bo', alpha=0.4, markersize=4, label='Class 0')
axes[1, 0].plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.4, markersize=4, label='Class 1')
axes[1, 0].plot(xx_true, yy_true, 'g-', linewidth=3, label='True Boundary')
axes[1, 0].contour(XX, YY, linear_predictions, levels=[0.5], colors='blue', linewidths=2, linestyles='--', label='Linear Boundary')
axes[1, 0].set_title('Linear Model vs True Boundary')
axes[1, 0].set_xlabel('X1')
axes[1, 0].set_ylabel('X2')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(x[y==0, 0], x[y==0, 1], 'bo', alpha=0.4, markersize=4, label='Class 0')
axes[1, 1].plot(x[y==1, 0], x[y==1, 1], 'ro', alpha=0.4, markersize=4, label='Class 1')
axes[1, 1].plot(xx_true, yy_true, 'g-', linewidth=3, label='True Boundary')
axes[1, 1].contour(XX, YY, mlp_predictions, levels=[0.5], colors='red', linewidths=2, linestyles='--', label='MLP Boundary')
axes[1, 1].set_title('MLP Model vs True Boundary')
axes[1, 1].set_xlabel('X1')
axes[1, 1].set_ylabel('X2')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:

linear_model.eval()
mlp_model.eval()

with torch.no_grad():
    linear_train_pred = (linear_model(X_tensor) > 0.5).float().cpu().numpy().flatten()
    mlp_train_pred = (mlp_model(X_tensor) > 0.5).float().cpu().numpy().flatten()

linear_errors = np.sum(linear_train_pred != y)
mlp_errors = np.sum(mlp_train_pred != y)

print("\n" + "="*60)
print("CLASSIFICATION ERROR ANALYSIS / 分類誤差解析")
print("="*60)
print(f"Linear Model - Training Errors: {linear_errors}/{len(y)} ({linear_errors/len(y)*100:.1f}%)")
print(f"MLP Model    - Training Errors: {mlp_errors}/{len(y)} ({mlp_errors/len(y)*100:.1f}%)")
print(f"線形モデル - 訓練誤差: {linear_errors}/{len(y)} ({linear_errors/len(y)*100:.1f}%)")
print(f"MLPモデル  - 訓練誤差: {mlp_errors}/{len(y)} ({mlp_errors/len(y)*100:.1f}%)")
print("="*60)

print("\nTutorial completed! / チュートリアル完了！")
print("Key insights / 主要な洞察:")
print("- Linear models struggle with nonlinear boundaries / 線形モデルは非線形境界に苦戦")
print("- MLPs can learn complex decision boundaries / MLPは複雑な決定境界を学習可能")
print("- PyTorch provides flexible neural network implementation / PyTorchは柔軟なニューラルネットワーク実装を提供")
